# Test de segmentation automatique des mots sur des données bruitées

On essaye de reconstituer automatiquement les mots agglutinés (_word segmentation_) sur des données du projet LECTAUREP, en utilisant le portage de la librairie SymSpell en python : https://github.com/mammothb/symspellpy

Grâce à l'issue suivante : https://github.com/wolfgarbe/SymSpell/issues/91, nous savons que nous pouvons utiliser un dictionnaire français pour utiliser la librairie sur des données en langue française, ainsi qu'utiliser un dictionnaire de bigrammes en langue française.

## Récupération d'un dictionnaire de bi-grammes français

On récupère le [dictionnaire de bi-grammes français](https://github.com/gturret/frenchngrams/blob/master/2gramstop10k.csv) disponibles sur le repository github suivant : https://github.com/gturret/frenchngrams

Etant au format CSV, il est nécessaire de convertir ce dictionnaire au format texte pour l'exploiter avec SymSpell, on utilise donc le script suivant :

In [6]:
import csv
import os

def convert_csv_to_txt(csv_source):
    data_dict = {}

    with open(csv_source, 'r', encoding='utf-8') as fh:
        file_name = os.path.basename(csv_source).strip('.csv')
        csv_file_content = csv.reader(fh, delimiter=',')
        # next(csv_file_content)

        with open(f'ngram_dictionaries/{file_name}.txt', 'w') as txt_file:
            for row in csv_file_content:
                print(row)
                txt_file.write(row[0])
                txt_file.write(' ')
                txt_file.write(row[1])
                txt_file.write('\n')

    return data_dict

new_txt_file = convert_csv_to_txt('ngram_dictionaries/2gramstop10k.csv')

['de la', '132940']
['à la', '56794']
['et de', '37743']
['dans la', '30090']
['que je', '28965']
['dans le', '26317']
['de son', '25492']
['je ne', '24297']
['dans les', '23172']
['et les', '22685']
['que le', '22135']
['de sa', '20853']
['et le', '20554']
['et la', '20000']
['que la', '19418']
['tous les', '19273']
['que les', '19027']
['il y', '18402']
['sur le', '18014']
['de ses', '16960']
['de ce', '16801']
['ce que', '16792']
['sur la', '16446']
['ce qui', '16378']
['que vous', '15658']
['il ne', '15143']
['il est', '15088']
['et il', '14218']
['de cette', '14192']
['de ces', '13667']
['toutes les', '13111']
['et qui', '13047']
['par la', '12722']
['sur les', '12639']
['je vous', '12555']
['y a', '12448']
['il se', '12438']
['que nous', '12112']
['et que', '11893']
['dans un', '11869']
['et des', '11638']
['le plus', '11468']
['à son', '11362']
['par le', '11099']
['qui se', '11047']
['et à', '10956']
['par les', '10734']
['qui ne', '10518']
["n'est pas", '10469']
['m de', '1029

Au format texte, toutes les lignes ont le format suivant :

* [token 1] [token 2] [fréquence]

## Test de SymSpellpy

On teste SymSpellpy, et notamment la méthode _lookup_compound_. Voir [documentation](https://symspellpy.readthedocs.io/en/latest/examples/lookup_compound.html?highlight=lookup_compound).

In [7]:
from symspellpy import SymSpell

# On instancie l'objet SymSpell en indiquant la distance d'édition maximale
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# On charge le dictionnaire français
sym_spell.load_dictionary("ngram_dictionaries/fr-100k.txt", term_index=0, count_index=1)
# On charge le dictionnaire des 10 000 bigrammes les plus fréquents dans la langue française
sym_spell.load_bigram_dictionary("ngram_dictionaries/2gramstop10k.txt", term_index=0, count_index=2)

# On stocke une chaine de caractère dans input_term pour tester
# Chaîne de caractère correspondant à des données bruitées en sortie d'HTR d'une page de répertoire de notaire
input_term = ("Lavignolle (rectificative parherman) etrequistion derertificat depropriété")

suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2, ignore_non_words=True, transfer_casing=True)

# On affiche les termes suggérés et la distance d'édition
for suggestion in suggestions:
    print(suggestion)

La vignoble rectificative par hermann et réquisition d certificat approprie te, 14, 0


Comme nous pouvons l'observer, le résultat n'est pas entièrement satisfaisant, notamment à cause de la surcorrection du nom de famille au début de la chaîne de caractère, et de l'échec de la reconstitution de "derertificat depropriété" en "certificat de propriété". De plus, on observe la disparition des parenthèses pour une raison inconnue jusque-là.

"etrequistion" a cependant bien été reconstitué en "et réquisition", et "parherman" en "par herman".

Peut-être que ces performances pourraient être améliorées par l'utilisation d'un dictionnaire de bigrammes constitués à partir des données des répertoires de notaires ? 